In [1]:
import os
from lib.train_AI_lib import *
import cv2
import numpy as np

In [2]:
os.chdir('..')

In [3]:
def createMask(bboxes, imgRes=(1024, 2014)):
    globalMask = np.zeros(imgRes, dtype=bool)
    for bbox in bboxes:
        oneMask = np.zeros(imgRes, dtype=bool)
        bbox = [int(number) for number in bbox]
        # Don't ask me why this flip works, numpy and cv2 take images in differently and it's a pain to try and figure out the transformations    
        oneMask[bbox[0]:(bbox[0]+bbox[2]), imgRes[1]-(bbox[1]+bbox[3]):imgRes[1]-bbox[1]] = np.ones((bbox[2], bbox[3]), dtype=bool)
        globalMask = np.bitwise_or(globalMask, oneMask)
    globalMask = np.rot90(globalMask, 1)
    return(globalMask)

def genBBoxImages(dictPath='saved/splitData', inPath='data/working-wheat-data/train', oPath='saved/bboxMasked'):
    try: os.makedirs(oPath)  # Make the requested oPath
    except FileExistsError: None
    except: print("error creating folder {}".format(oPath)); return(0)
    else: None
    
    trainData = torch.load(dictPath+"/trainData")
    valData   = torch.load(dictPath+"/valData"  )
    testData  = torch.load(dictPath+"/testData" )
    trainData.update(valData); trainData.update(testData)
    imgPath   = inPath+'/'
    keyList   = list(trainData.keys())

    for i, imgName in enumerate(keyList):
        img  = cv2.imread(imgPath+imgName)
        bbox = trainData[imgName]
        mask = createMask(bbox, np.shape(img)[:-1])
        img[:, :, 0] = np.multiply(img[:, :, 0], mask)
        img[:, :, 1] = np.multiply(img[:, :, 1], mask)
        img[:, :, 2] = np.multiply(img[:, :, 2], mask)

        if i%100==0: print("Converted {:.2f}%".format(100*i/len(keyList)))
        cv2.imwrite(oPath+'/'+imgName, img)

In [4]:
#genBBoxImages()

In [5]:
class autEncA(nn.Module):
    def __init__(self, name):
        super(autEncA, self).__init__()
        self.name = name
        self.encoder = nn.Sequential( # like the Composition layer you built
            nn.Conv2d(3, 5, 3, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(5, 10, 3, stride=2, padding=1),
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(10, 5, 3, stride=2, padding=1, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(5, 3, 3, stride=2, padding=1, output_padding=1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [6]:
batchsize=64; epoch=10; lr=0.0001
net = autEncA('autEncA'); net.cuda()
trainLoader, valLoader, testLoader = loadData(batchsize, mode='auto', altArg={'path':'saved/bboxMasked'})
_ = trainNet(net, [trainLoader, valLoader], batchsize, epoch, lr, trainType='AutoEnc')

RuntimeError: CUDA out of memory. Tried to allocate 320.00 MiB (GPU 0; 11.17 GiB total capacity; 10.53 GiB already allocated; 207.75 MiB free; 10.66 GiB reserved in total by PyTorch)